## IBGE Data Analysis Project

IBGE is the main source for all kinds os statistical data in Brazil. This repository is an exploration of interesting socioeconomic data as well as concepts like Data visualization, REST API abstraction and Test driven development applied to Data Science.

In [1]:
import urllib3
#import json
import matplotlib.pyplot as plot
import numpy
import pandas
from ibge_lib import *


In [2]:
http = urllib3.PoolManager()
http_fn = lambda url: http.request('GET', url)

query_url = aggregates_url()
researches = query_ibge(query_url, http_fn)

 ### Picking an aggregate

 IBGE's API needs the exact data aggregate code in order to supply any useful data. Therefore, we need to ask for the list of aggregates and search for the code of the aggregate we want. Luckly, these are grouped by research. In this case, we want the National Household Sampling Research (PNAD).

In [3]:
format_fn = lambda x: f"{x['id']}: {x['nome']}"
list(map(format_fn, researches))

['CL: Cadastro Central de Empresas',
 'CA: Censo Agropecuário',
 'ME: Censo Comum do Mercosul, Bolívia e Chile',
 'CD: Censo Demográfico',
 'CM: Contagem da População ',
 'DT: Contas de Espécies Ameaçadas ',
 'DU: Contas de Extensão dos Ecossistemas',
 'C4: Contas Econômicas Ambientais da Água',
 'DZ: Contas Econômicas Ambientais de Energia: produtos da biomassa',
 'SU: Contas Nacionais Anuais',
 'ST: Contas Nacionais Trimestrais',
 'DE: Demografia das Empresas e Estatísticas de Empreendedorismo',
 'XF: Estimativas de População',
 'FP: Fundações Privadas e Associações Sem Fins Lucrativos',
 'IU: Indicadores de Desenvolvimento Sustentável',
 'IL: Índice de Preços ao Consumidor em Real',
 'IR: Índice de Preços ao Produtor',
 'IJ: Índice de Reajuste do Salário Mínimo',
 'PC: Índice Nacional de Preços ao Consumidor',
 'IA: Índice Nacional de Preços ao Consumidor Amplo',
 'IQ: Índice Nacional de Preços ao Consumidor Amplo 15',
 'IG: Índice Nacional de Preços ao Consumidor Amplo Especial',
 

In [7]:
pnad_id = 'B5'
filter_pnad_fn = lambda x: x['id'] == pnad_id
pnad_data = next(filter(filter_pnad_fn, researches)) # 'next' gets the first element, in this case
#pnad_data

In [8]:
filtered_aggr = filter_aggregates(pnad_data, 'rendimento domiciliar per capita')
format_fn = lambda aggr: f"{aggr['id']}: {aggr['nome']}"
list(map(format_fn, filtered_aggr))

['7530: Distribuição da massa do rendimento mensal real domiciliar per capita, a preços médios do ano, por classes acumuladas de percentual das pessoas em ordem crescente de rendimento domiciliar per capita',
 '7527: Distribuição da massa do rendimento mensal real domiciliar per capita, a preços médios do ano, por classes simples de percentual das pessoas em ordem crescente de rendimento domiciliar per capita',
 '6411: Domicílios em que nenhum morador viajou no período de referência dos últimos três meses, por classes de rendimento nominal domiciliar per capita e motivo de nenhum morador ter viajado (2019)',
 '7229: Domicílios, por classes de rendimento nominal domiciliar per capita e ocorrência de viagem de moradores no período de referência dos últimos três meses',
 '7435: Índice de Gini do rendimento domiciliar per capita, a preços médios do ano',
 '7438: Limites superiores das classes de percentual das pessoas em ordem crescente de rendimento domiciliar per capita, a preços médios 

In [ ]:
PERCENTILES = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99, 100]
LOCATIONS = ['BR']

#Boundries query arguments
BOUND_AGGR = '7438'
BOUND_VAR = ['10769']
BOUND_CLASSIF = ['1019[49259,49260,49261,49262,49263,49264,49265,49266,49267,49268,49269,49270]']

#Income distribution query arguments
DISTRIBUTION_AGGR = '7530'
DISTRIBUTION_VAR = ['10826']
DISTRIBUTION_CLASSIF = ['1042[49271,49272,49273,49274,49275,49276,49277,49278,49279,49280,49281,49282]']


In [ ]:
income_classes_url = variables_url(aggregate=BOUND_AGGR, variables=BOUND_VAR, locations=LOCATIONS, classifications=BOUND_CLASSIF, periods=['2019'])
income_classes = query_ibge(income_classes_url, http_fn)
income_classes = [ float(percentile['series'][0]['serie']['2019']) for percentile in income_classes ]

cumm_dist_url = variables_url(DISTRIBUTION_AGGR, DISTRIBUTION_VAR, LOCATIONS, DISTRIBUTION_CLASSIF)
cumm_dist = query_ibge(cumm_dist_url, http_fn)
cumm_dist = [ float(percentile['series'][0]['serie']['2019']) for percentile in cumm_dist ]
cumm_dist.append(100.0)

income_dist = [cumm_dist[0]]
for i in range(1, len(cumm_dist)):
    income_dist.append(cumm_dist[i] - cumm_dist[i-1])

In [ ]:
X = numpy.array(PERCENTILES)
widths = numpy.array([-5,  -5, -10, -10, -10, -10, -10, -10, -10, - 10,  -5,  -4,   -1])
Y = numpy.array(income_dist)/(-widths)

fig, ax = plot.subplots()
ax.bar(X, Y, align='edge', width=widths)